## ***Machine Learning***
El modelo deberá tener una relación `ítem-ítem`, esto es se toma un item, en base a que tan similar es ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello recomendamos aplicaremos la similitud del coseno.

**Comenzamos trayendo los datos, las librerias y creando nuestro sistema de recomendación usando un filtro-colaborativo de `item-item`. Entonces...**
<img src = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTqF0wkErtJDCV5QzwoPO_9sdEA7nxQdk5EJA&usqp=CAU">

### 1. Importamos  librerías

In [1]:
# Importtamos librerias
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity

### 1.1 Carga inicial de los datos

In [17]:
games   = pd.read_parquet("Data/steam_games.parquet")
reviews = pd.read_parquet("Data/steam_reviews.parquet")
items   = pd.read_parquet("Data/steam_items.parquet")

In [18]:
# Convertir en Dataframe 
df_games   = pd.DataFrame(games)
df_reviews = pd.DataFrame(reviews)
df_items   = pd.DataFrame(items)

### 1.2 Combinamos los datos 

In [19]:
print(df_games.columns)
print(df_reviews.columns)
print(df_items.columns)

Index(['name', 'url', 'reviews_url', 'price', 'item_id', 'Year', 'genres',
       'tags', 'specs', 'developer'],
      dtype='object')
Index(['user_id', 'user_url', 'posted', 'item_id', 'recommend',
       'sentiment_analysis'],
      dtype='object')
Index(['user_id', 'items_count', 'steam_id', 'user_url', 'item_id',
       'item_name', 'playtime_forever', 'playtime_2weeks'],
      dtype='object')


Necesitamos una columna en cada DataFrame, son: en df_games es item_id, en df_reviews es sentiment_analysis y en df_items es user_id

In [20]:
print(df_games.shape)
print(df_items.shape)
print(df_reviews.shape)


(1977302, 10)
(3246378, 8)
(58431, 6)


In [21]:
df_games.head()

,name,url,reviews_url,price,item_id,Year,genres,tags,specs,developer
0,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140,2018,Action,Strategy,Single-player,kotoshiro
1,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140,2018,Action,Action,Single-player,kotoshiro
2,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140,2018,Action,Indie,Single-player,kotoshiro
3,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140,2018,Action,Casual,Single-player,kotoshiro
4,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140,2018,Action,Simulation,Single-player,kotoshiro


In [14]:
df_games['item_id'].unique().sum()


-409341093